In [1]:
import pandas as pd
import numpy as np

In [120]:
df = pd.read_csv("league_record.csv")

In [3]:
pd.set_option('display.max_columns', None)

In [121]:
df

,league_id,week,hour,minutes,HT,AT,HLS,ALS,HL2S,AL2S,HL3S,AL3S,HL4S,AL4S,HL5S,AL5S,hl_stat,al_stat,hl2_stat,al2_stat,hl3_stat,al3_stat,h_wins,a_wins,h_draws,a_draws,h_loss,a_loss,h_gf,a_gf,h_ga,a_ga,h_gd,a_gd,ht_points,at_points,ht_pos,at_pos
0,6095,1,12,52,EVE,MNC,1,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,1,2,2,1,-1,1,0,3,9,1
1,6095,1,12,52,BRN,TOT,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1,1,0,-1,1,0,3,7,2
2,6095,1,12,52,BRI,FUL,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,1,0,0,1,1,-1,3,0,1,12
3,6095,1,12,52,LEI,WHU,2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,2,1,1,2,1,-1,3,0,2,19
4,6095,1,12,52,WOL,ARS,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,1,1,6,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4555,6171,38,14,22,TOT,CHE,0,0,0,1,2,2,3,1,2,1,1,1,0,2,1,2,19,17,7,13,12,8,64,45,44,30,20,15,64,64,7,6
4556,6171,38,14,22,FUL,MNC,1,0,1,1,2,4,0,3,1,1,2,0,0,1,0,2,7,25,9,8,22,5,37,75,61,33,-24,42,30,83,19,2
4557,6171,38,14,22,WHU,BOU,3,0,1,0,1,2,2,0,3,0,2,0,0,1,1,1,24,8,9,7,5,23,71,39,35,76,36,-37,81,31,3,18
4558,6171,38,14,22,ARS,WOL,2,0,2,2,0,1,3,0,3,1,2,0,2,1,0,2,23,11,5,12,10,15,71,46,49,50,22,-4,74,45,4,13


#### Home Win Model

In [122]:
# Home win prediction
df_home = df.copy()
df_home = df_home[df_home['week'] > 4]  # remove rows with incomplete values
df_home = df_home.drop(["league_id", "ALS", "HT", "AT", "hl_stat", "al_stat"], axis=1)  # remove unnecessary features
df_home = df_home.reset_index(drop=True)  # reset the index values
df_home

,week,hour,minutes,HLS,HL2S,AL2S,HL3S,AL3S,HL4S,AL4S,HL5S,AL5S,hl2_stat,al2_stat,hl3_stat,al3_stat,h_wins,a_wins,h_draws,a_draws,h_loss,a_loss,h_gf,a_gf,h_ga,a_ga,h_gd,a_gd,ht_points,at_points,ht_pos,at_pos
0,5,13,0,0,2,1,2,0,1,2,1,2,2,1,2,0,2,0,1,4,2,1,6,5,8,10,-2,-5,7,4,6,13
1,5,13,0,2,0,3,5,0,0,0,2,2,0,2,2,0,3,1,0,1,2,3,9,6,4,8,5,-2,9,4,4,15
2,5,13,0,1,1,1,1,0,0,1,3,2,0,0,1,0,1,2,2,1,2,2,6,5,9,4,-3,1,5,7,12,9
3,5,13,0,1,0,1,0,4,0,3,0,3,0,2,1,2,0,5,2,0,3,0,1,15,7,4,-6,11,2,15,19,1
4,5,13,0,1,2,2,4,1,2,2,2,1,2,2,2,0,3,3,0,0,2,2,11,8,8,6,3,2,9,9,3,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4075,38,14,22,0,0,1,2,2,3,1,2,1,0,2,1,2,19,17,7,13,12,8,64,45,44,30,20,15,64,64,7,6
4076,38,14,22,1,1,1,2,4,0,3,1,1,0,1,0,2,7,25,9,8,22,5,37,75,61,33,-24,42,30,83,19,2
4077,38,14,22,3,1,0,1,2,2,0,3,0,0,1,1,1,24,8,9,7,5,23,71,39,35,76,36,-37,81,31,3,18
4078,38,14,22,2,2,2,0,1,3,0,3,1,2,1,0,2,23,11,5,12,10,15,71,46,49,50,22,-4,74,45,4,13


### 1. One-Hot Encoding:

In [123]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder

df_temp1 = df_home.copy()

# create a one-hot encoder object
encoder = OneHotEncoder(sparse_output=False)

# fit the encoder to the categorical columns
encoder.fit(df_temp1[['hl2_stat', 'al2_stat', 'hl3_stat', 'al3_stat']])

# transform the categorical columns into one-hot encoded columns
one_hot_encoded = pd.DataFrame(encoder.transform(df_temp1[['hl2_stat', 'al2_stat', 'hl3_stat', 'al3_stat']]))

# concatenate the one-hot encoded columns with the original data frame
df_onehot = pd.concat([df_temp1, one_hot_encoded], axis=1)

# drop the original categorical columns
df_onehot.drop(['hl2_stat', 'al2_stat', 'hl3_stat', 'al3_stat'], axis=1, inplace=True)
df_onehot = df_onehot.dropna()

# rename columns to string names
df_onehot.columns = df_onehot.columns.astype(str)

df_onehot.shape

(4080, 40)

In [134]:
df_ordinal.tail(3)

,week,hour,minutes,HLS,HL2S,AL2S,HL3S,AL3S,HL4S,AL4S,HL5S,AL5S,h_wins,a_wins,h_draws,a_draws,h_loss,a_loss,h_gf,a_gf,h_ga,a_ga,h_gd,a_gd,ht_points,at_points,ht_pos,at_pos,hl2_stat_enc,al2_stat_enc,hl3_stat_enc,al3_stat_enc
4077,38,14,22,3,1,0,1,2,2,0,3,0,24,8,9,7,5,23,71,39,35,76,36,-37,81,31,3,18,0.0,1.0,1.0,1.0
4078,38,14,22,2,2,2,0,1,3,0,3,1,23,11,5,12,10,15,71,46,49,50,22,-4,74,45,4,13,2.0,1.0,0.0,2.0
4079,38,14,22,2,0,2,0,1,0,0,0,4,12,15,13,7,13,16,47,55,53,60,-6,-5,49,52,11,9,0.0,2.0,0.0,0.0


### 2. Ordinal Encoding:

In [124]:
import pandas as pd
from sklearn.preprocessing import OrdinalEncoder

df_temp2 = df_home.copy()

# create an ordinal encoder object with specified categories
encoder = OrdinalEncoder(categories=[sorted(df_temp2[col].unique()) for col in ['hl2_stat', 'al2_stat', 'hl3_stat', 'al3_stat']])

# fit the encoder to the categorical columns
encoder.fit(df_temp2[['hl2_stat', 'al2_stat', 'hl3_stat', 'al3_stat']])

# transform the categorical columns into ordinal encoded columns
ordinal_encoded = pd.DataFrame(encoder.transform(df_temp2[['hl2_stat', 'al2_stat', 'hl3_stat', 'al3_stat']]), \
columns=['hl2_stat_enc', 'al2_stat_enc', 'hl3_stat_enc', 'al3_stat_enc'])

# concatenate the ordinal encoded columns with the original data frame
df_ordinal = pd.concat([df_temp2, ordinal_encoded], axis=1)

# drop the original categorical columns
df_ordinal.drop(['hl2_stat', 'al2_stat', 'hl3_stat', 'al3_stat'], axis=1, inplace=True)

df_ordinal.shape

(4080, 32)

In [133]:
df_ordinal.head()

,week,hour,minutes,HLS,HL2S,AL2S,HL3S,AL3S,HL4S,AL4S,HL5S,AL5S,h_wins,a_wins,h_draws,a_draws,h_loss,a_loss,h_gf,a_gf,h_ga,a_ga,h_gd,a_gd,ht_points,at_points,ht_pos,at_pos,hl2_stat_enc,al2_stat_enc,hl3_stat_enc,al3_stat_enc
0,5,13,0,0,2,1,2,0,1,2,1,2,2,0,1,4,2,1,6,5,8,10,-2,-5,7,4,6,13,2.0,1.0,2.0,0.0
1,5,13,0,2,0,3,5,0,0,0,2,2,3,1,0,1,2,3,9,6,4,8,5,-2,9,4,4,15,0.0,2.0,2.0,0.0
2,5,13,0,1,1,1,1,0,0,1,3,2,1,2,2,1,2,2,6,5,9,4,-3,1,5,7,12,9,0.0,0.0,1.0,0.0
3,5,13,0,1,0,1,0,4,0,3,0,3,0,5,2,0,3,0,1,15,7,4,-6,11,2,15,19,1,0.0,2.0,1.0,2.0
4,5,13,0,1,2,2,4,1,2,2,2,1,3,3,0,0,2,2,11,8,8,6,3,2,9,9,3,5,2.0,2.0,2.0,0.0


### 3. Target Encoding:

In [126]:
import pandas as pd
import category_encoders as ce

df_temp4 = df_home.copy()

# create a target encoder object
encoder = ce.TargetEncoder(cols=['hl2_stat', 'al2_stat', 'hl3_stat', 'al3_stat'])

# fit the encoder to the categorical columns and target variable
encoder.fit(df_temp4[['hl2_stat', 'al2_stat', 'hl3_stat', 'al3_stat']], df_temp4['HLS'])

# transform the categorical columns into target encoded columns
target_encoded = encoder.transform(df_temp4[['hl2_stat', 'al2_stat', 'hl3_stat', 'al3_stat']])

# concatenate the target encoded columns with the original data frame
df_encoded = pd.concat([df_temp4, target_encoded], axis=1)

# drop the original categorical columns
df_encoded.drop(['hl2_stat', 'al2_stat', 'hl3_stat', 'al3_stat'], axis=1, inplace=True)

df_encoded.shape

(4080, 28)

# Model Evaluation

In [128]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.metrics import mean_squared_error, accuracy_score

def train_random_forest(dataset, target_var, test_size=0.2, random_state=42, **kwargs):
    """
    Train a random forest model on the given dataset and target variable.

    Args:
        dataset (pandas.DataFrame): The dataset to use for training and testing.
        target_var (str): The name of the target variable to predict.
        test_size (float, optional): The proportion of the data to use for testing. Defaults to 0.2.
        random_state (int, optional): The random seed to use for reproducibility. Defaults to 42.
        **kwargs: Other keyword arguments to pass to the RandomForestRegressor or RandomForestClassifier constructor.

    Returns:
        (float, float): A tuple containing the root mean squared error and accuracy of the model on the test set.
    """
    # Split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(dataset.drop(target_var, axis=1), dataset[target_var], test_size=test_size, random_state=random_state)
    
    # Train a random forest model on the training set
    if isinstance(y_train.iloc[0], str):
        is_classification = True
        model = RandomForestClassifier(random_state=random_state, **kwargs)
    else:
        is_classification = False
        model = RandomForestRegressor(random_state=random_state, **kwargs)
    model.fit(X_train, y_train)
    
    # Make predictions on the test set
    y_pred = model.predict(X_test)
    
    # Calculate the RMSE and accuracy of the model on the test set
    if is_classification:
        accuracy = accuracy_score(y_test, y_pred)
    else:
        accuracy = accuracy_score(y_test.round(), y_pred.round())
    
    rmse = mean_squared_error(y_test, y_pred, squared=False)
    
    return rmse, accuracy

In [135]:
datasets = [df_home]
for data in datasets:   
    rmse, accuracy = train_random_forest(data, 'HLS', n_estimators=100)
    print(f'RMSE: {rmse}, Accuracy: {accuracy}')

RMSE: 1.1215452946890803, Accuracy: 0.37254901960784315


In [130]:
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.metrics import mean_squared_error, accuracy_score


def cross_validate_random_forest(dataset, target_var, n_splits=5, random_state=42, **kwargs):
    """
    Perform k-fold cross validation on a random forest model on the given dataset and target variable.

    Args:
        dataset (pandas.DataFrame): The dataset to use for training and testing.
        target_var (str): The name of the target variable to predict.
        n_splits (int, optional): The number of folds to use for cross validation. Defaults to 5.
        random_state (int, optional): The random seed to use for reproducibility. Defaults to 42.
        **kwargs: Other keyword arguments to pass to the RandomForestRegressor or RandomForestClassifier constructor.

    Returns:
        (float, float): A tuple containing the mean root mean squared error and mean accuracy of the model across all folds for the appropriate task (regression or classification).
    """
    # Initialize the KFold cross validation object
    kfold = KFold(n_splits=n_splits, shuffle=True, random_state=random_state)

    # Initialize lists to store the RMSE and accuracy values for each fold
    rmse_scores = []
    accuracy_scores = []

    # Determine whether the target variable is categorical or numeric
    if isinstance(dataset[target_var].iloc[0], str):
        is_classification = True
    else:
        is_classification = False

    # Loop over each fold
    for train_idx, test_idx in kfold.split(dataset):
        # Split the data into training and testing sets for the current fold
        X_train, X_test = dataset.drop(target_var, axis=1).iloc[train_idx], dataset.drop(target_var, axis=1).iloc[test_idx]
        y_train, y_test = dataset[target_var].iloc[train_idx], dataset[target_var].iloc[test_idx]

        # Train a random forest model on the training set
        if isinstance(y_train.iloc[0], str):
            is_classification = True
            model = RandomForestClassifier(random_state=random_state, **kwargs)
        else:
            is_classification = False
            model = RandomForestRegressor(random_state=random_state, **kwargs)
        model.fit(X_train, y_train)
        
        # Make predictions on the test set
        y_pred = model.predict(X_test)
        
        # Calculate the RMSE and accuracy of the model on the test set
        if is_classification:
            accuracy = accuracy_score(y_test, y_pred)
        else:
            accuracy = accuracy_score(y_test.round(), y_pred.round())
        
        rmse = mean_squared_error(y_test, y_pred, squared=False)
        
        # Append the RMSE and accuracy values for the current fold to the lists
        rmse_scores.append(rmse)
        accuracy_scores.append(accuracy)

    # Calculate the mean RMSE and accuracy across all folds
    if is_classification:
        mean_accuracy = sum(accuracy_scores) / n_splits
        mean_rmse = None
    else:
        mean_rmse = sum(rmse_scores) / n_splits
        mean_accuracy = accuracy_score(y_test.round(), y_pred.round())

    return mean_rmse, mean_accuracy

In [131]:
rmse, accuracy = cross_validate_random_forest(df_home, "HLS", n_splits=100, random_state=42)
print(f'RMSE: {rmse}, Accuracy: {accuracy}')

RMSE: 1.033632532316481, Accuracy: 0.375
RMSE: 1.0352467521128554, Accuracy: 0.375
RMSE: 1.0344052955166947, Accuracy: 0.35


In [ ]:
# RMSE: 1.037260024806251, Accuracy: 0.45588235294117646    
# RMSE: 1.0395867421206864, Accuracy: 0.4411764705882353
# RMSE: 1.0394407662064982, Accuracy: 0.4117647058823529

# RMSE: 1.0291323227675842, Accuracy: 0.29411764705882354
# RMSE: 1.0306463633341978, Accuracy: 0.3235294117647059
# RMSE: 1.032703152907061, Accuracy: 0.2647058823529412

# RMSE: 1.0311636872784198, Accuracy: 0.40540540540540543
# RMSE: 1.0324646077714403, Accuracy: 0.40540540540540543
# RMSE: 1.0303426207783912, Accuracy: 0.40540540540540543